# Fresh take

Trying to get this right.  2024/02/18

## Setup an inventory

In [196]:
import os
import yaml
import copy
import shutil
import numpy as np
import pandas as pd
import importlib
import NewDev as ND
import helperFunctions as hf
importlib.reload(ND)
importlib.reload(hf)
                          
dbpth = os.path.join(os.getcwd(),'test')
try:
    shutil.rmtree(dbpth)
except:
    pass
# os.mkdir(dbpth)
ND.siteInventory(projectPath=dbpth)
ND.siteInventory(projectPath = dbpth,siteID='SCL',description='Swiss Cheese Lake Eddy Covariance Site',
                latitude='N 69 13.5850',longitude='W 135 15.1144',
                PI = 'June Skeeter, Peter Morse',dateEstablished='2024-07-20')
# ND.siteInventory(projectPath = dbpth,siteID='FIL',description='Fish Island Eddy Covariance Site',
#                   latitude ='69°22′20.20″N', longitude = '134°52′51.92″W',
#                 PI = 'June Skeeter', dateEstablished='2017-06-21')
# ND.siteInventory(projectPath = dbpth,siteID='ILL',description='Illisarvik Research Site',
#                   latitude ='69°28′047.500" N', longitude = '34° 50′ 18.700"W',
#                 PI = 'Chris Burn', dateEstablished='1978-08-01')
# list(db.siteInventory.siteInventory.keys())[0]


# siteFields = set(self.__dataclass_fields__) - {f for base in siteRecord.__bases__ if hasattr(base,'__dataclass_fields__') for f in base.__dataclass_fields__}


siteInventory(siteID='SCL', measurementType=None, loggerID=None)

In [170]:
# db._siteInventory(siteID='SCL',measurementID='EC',description='Eddy Covariance Station',
                # latitude='N 69 13.5850',longitude='W 135 15.1144',
                # PI = 'June Skeeter, Peter Morse',dateEstablished='2024-07-20' )
# db._siteInventory(siteID='SCL',measurementID='TS',description='Wet Sedge Meadow',
#                   latitude='N 69 13.5521',longitude='W 135 15.1350',overwrite=True)

## File Management

In [195]:
import time
import rawDataFile as rd
importlib.reload(rd)
source = r'..\SCL_2024'
T1 = time.time()

t = rd.measurementInventory(projectPath=dbpth,siteID='SCL',measurementType='TS_Profile',loggerID='SSM',fileType='HoboCSV')
print(t.ID)
# t = rd.measurementInventory(projectPath=dbpth,siteID='SCL',measurementType='TS_Profile',loggerID='WSM',fileType='HoboCSV')
# t = rd.fileInventory(projectPath=dbpth,siteID='SCL',ID='TS_Profile-SSM',source=source)
# fi = rd.fileInventory(projectPath=dbpth,siteID='SCL',measurementType='TS_Profile',loggerID='WSM',source=source,matchPattern='WSM')
# fi = rd.fileInventory(projectPath=dbpth,siteID='SCL',measurementType='TS_Profile',loggerID='SSM',source=source,matchPattern='SSM')
# fi = rd.fileInventory(projectPath=dbpth,siteID='SCL',measurementType='Eddy Covariance',loggerID='CR1000x',source=source,matchPattern='Flux',ext='dat')
# fi = rd.fileInventory(projectPath=dbpth,siteID='SCL',measurementType='Meteorological',loggerID='CR1000x',source=source,matchPattern='Met_',ext='dat')
# fi = rd.fileInventory(projectPath=dbpth,siteID='SCL',measurementType='Metadata',loggerID='CR1000x',source=source,matchPattern='Meta',ext='dat')
# print(t.record)
print(time.time()-T1)


SCL-TS_Profile-SSM
0.012539386749267578


In [102]:
print(rd.measurementInventory(projectPath=dbpth,siteID='SCL',measurementType='A').__dataclass_fields__)

{'projectPath': Field(name='projectPath',type=<class 'str'>,default=<dataclasses._MISSING_TYPE object at 0x00000227A17821D0>,default_factory=<dataclasses._MISSING_TYPE object at 0x00000227A17821D0>,init=True,repr=False,hash=None,compare=True,metadata=mappingproxy({}),kw_only=True,_field_type=_FIELD), 'overwrite': Field(name='overwrite',type=<class 'bool'>,default=False,default_factory=<dataclasses._MISSING_TYPE object at 0x00000227A17821D0>,init=True,repr=False,hash=None,compare=True,metadata=mappingproxy({}),kw_only=True,_field_type=_FIELD), 'verbose': Field(name='verbose',type=<class 'bool'>,default=True,default_factory=<dataclasses._MISSING_TYPE object at 0x00000227A17821D0>,init=True,repr=False,hash=None,compare=True,metadata=mappingproxy({}),kw_only=True,_field_type=_FIELD), 'metadataFile': Field(name='metadataFile',type=<class 'dict'>,default=<dataclasses._MISSING_TYPE object at 0x00000227A17821D0>,default_factory=<function database.<lambda> at 0x00000227B5DB5940>,init=True,repr=

In [59]:
d = {'a':'b',
     'c':'d'}

list(d.items())[::-1]

[('c', 'd'), ('a', 'b')]

In [126]:
# fn
import os
import rawDataFile as rawDataFile
importlib.reload(rawDataFile)
# fn = fi['SCL']['TS_Profile-WSM']['c:\\Users\\jskeeter\\gsc-permafrost\\SCL_2024']['sourceFiles'][0]
# fn = os.path.join('c:\\Users\\jskeeter\\gsc-permafrost\\SCL_2024',fn)
fn = r'c:\\Users\\jskeeter\\gsc-permafrost\\SCL_2024\\20240724\\20750527-SHSC.WSM.SGT.240720_240724readout.csv'
rd = rawDataFile.hoboCSV(projectPath=dbpth,source=fn,siteID='SCL',measurementType='TS_Profile',loggerID='WSM',
                         Metadata='test.yml')

with open('test.yml','w+') as f:
    yaml.safe_dump(rd.Metadata,f,sort_keys=False)

{'TS_1_1_1': {'variableName': 'TS', 'originalName': 'Temp, °C (LGR S/N: 20750527, SEN S/N: 20750527, LBL: 5)', 'ignore': False, 'unit': 'deg C', 'dtype': '<f8', 'variableDescription': 'Soil Temperature at 5 cm depth', 'vertical': 1, 'horizontal': 1, 'replicate': 1}, 'TS_2_1_1': {'variableName': 'TS', 'originalName': 'Temp, °C (LGR S/N: 20750527, SEN S/N: 20750527, LBL: 25)', 'ignore': False, 'unit': 'deg C', 'dtype': '<f8', 'variableDescription': 'Soil Temperature at 25 cm depth', 'vertical': 2, 'horizontal': 1, 'replicate': 1}, 'TS_3_1_1': {'variableName': 'TS', 'originalName': 'Temp, °C (LGR S/N: 20750527, SEN S/N: 20750527, LBL: 50)', 'ignore': False, 'unit': 'deg C', 'dtype': '<f8', 'variableDescription': 'Soil Temperature at 50 cm depth', 'vertical': 3, 'horizontal': 1, 'replicate': 1}, 'TS_4_1_1': {'variableName': 'TS', 'originalName': 'Temp, °C (LGR S/N: 20750527, SEN S/N: 20750527, LBL: 100)', 'ignore': False, 'unit': 'deg C', 'dtype': '<f8', 'variableDescription': 'Soil Temper

# Database

In [322]:
importlib.reload(ND)

# dbpth = os.path.join(os.getcwd(),'test2')
# # if os.path.isdir(dbpth):
# #     shutil.rmtree(dbpth)

# db = pyDB.database(projectPath=dbpth)
# # db = pyDB.database(projectPath=dbpth,
# #     # siteID=['SCL'],
# #     # Years=[str(y) for y in range(2024,2025)],
# #     # verbose=False
# #     )
# # db.
n = ND.rawDatabaseImport(projectPath=dbpth,measurementID ='test',dataIn=rd.Data,metadataIn=rd.Metadata)

n

_ {'ignore': True, 'name_in': '#', 'unit_in': None, 'safe_name': '_', 'dtype': '<i8', 'variableDescription': None}
Date_Time_GMT_00_00 {'ignore': True, 'name_in': 'Date Time, GMT+00:00', 'unit_in': None, 'safe_name': 'Date_Time_GMT_00_00', 'dtype': '|O', 'variableDescription': None}
Temp_C_5cm {'ignore': False, 'name_in': 'Temp, °C (5cm', 'unit_in': None, 'safe_name': 'Temp_C_5cm', 'dtype': '<f8', 'variableDescription': None}
Temp_C_25cm {'ignore': False, 'name_in': 'Temp, °C (25cm', 'unit_in': None, 'safe_name': 'Temp_C_25cm', 'dtype': '<f8', 'variableDescription': None}
Temp_C_50cm {'ignore': False, 'name_in': 'Temp, °C (50cm', 'unit_in': None, 'safe_name': 'Temp_C_50cm', 'dtype': '<f8', 'variableDescription': None}
Temp_C_100cm {'ignore': False, 'name_in': 'Temp, °C (100cm', 'unit_in': None, 'safe_name': 'Temp_C_100cm', 'dtype': '<f8', 'variableDescription': None}


rawDatabaseImport(projectPath='c:\\Users\\User\\GSC_Work\\EC_dataPipeline\\test', overwrite=False, verbose=True, logFile='Creating Database: 2025-03-01 21:22:42\n', metadataFile={'defaultFormat': {'data': {'dtype': 'float32', 'info': 'data are stored as 32-bit floating point numbers'}, 'POSIX_timestamp': {'dtype': 'float64', 'timezone': 'UTC', 'info': 'the POSIX_timestamp is stored as a 64-bit floating point number, representing the seconds elapsed since 1970-01-01 00:00 UTC time'}}, 'Date_created': '2025-03-01 21:22:42', 'Last_modified': '2025-03-01 21:22:42'}, measurementID='test', dataIn=                        _ Date_Time_GMT_00_00  Temp_C_5cm  Temp_C_25cm  \
2024-07-20 21:00:00     1   24/07/20 21:00:00       7.217        2.423   
2024-07-20 21:30:00     2   24/07/20 21:30:00      11.759        6.712   
2024-07-20 22:00:00     3   24/07/20 22:00:00      11.467        5.257   
2024-07-20 22:30:00     4   24/07/20 22:30:00      11.029        4.428   
2024-07-20 23:00:00     5   24/0

In [61]:
siteID,subsiteID='a'.split('-',1)

ValueError: not enough values to unpack (expected 2, got 1)

In [82]:
from dataclasses import dataclass, fields

@dataclass
class T1:
    a: str = "1"

@dataclass
class T2:
    b: str = "1"

@dataclass
class T3(T1, T2):
    c: str = "1"
    d: str = "d"

print(type(T3()).mro())
print(T3())



[<class '__main__.T3'>, <class '__main__.T1'>, <class '__main__.T2'>, <class 'object'>]
T3(b='1', a='1', c='1', d='d')
